# Install [Argo CD](https://argoproj.github.io/argo-cd/)

In [ ]:
source ../config.sh

kubectl config use-context ${CICD_CLUSTER}

echo
echo 'Create Argo CD namespace...'
kubectl create namespace argocd

echo
echo 'Install Argo CD on the CI/CD cluster...'
kubectl -n argocd apply -f https://raw.githubusercontent.com/argoproj/argo-cd/stable/manifests/install.yaml

echo
echo 'Fetch a Load Balanced IP for the Argo CD server...'
kubectl -n argocd patch svc argocd-server -p '{"spec": {"type": "LoadBalancer"}}'

### Wait for load balancer to be provisioned...

In [ ]:
source ../config.sh

kubectl config use-context ${CICD_CLUSTER}

kubectl -n argocd get svc argocd-server
nslookup `kubectl -n argocd get svc/argocd-server -o=jsonpath='{.status.loadBalancer.ingress[0].hostname}'`

### Finish configuring Argo CD

In [ ]:
source ../config.sh

kubectl config use-context ${CICD_CLUSTER}

ARGOCD_USERNAME=admin
ARGOCD_PASSWORD=`kubectl -n argocd get pods -l app.kubernetes.io/name=argocd-server -o name | cut -d'/' -f 2`

echo
echo 'Store the Argo CD host name...'
ARGOCD_HOST=`kubectl -n argocd get svc/argocd-server -o json | jq '.status.loadBalancer.ingress[0].hostname' -j`

argocd login ${ARGOCD_HOST} --name argocd --username ${ARGOCD_USERNAME} --password ${ARGOCD_PASSWORD} --insecure

argocd cluster add ${ALPHA_CLUSTER}
argocd cluster add ${BRAVO_CLUSTER}

echo
echo "Argo CD Host: $ARGOCD_HOST"
echo "Username: $ARGOCD_USERNAME"
echo "Password: $ARGOCD_PASSWORD"